## Imports

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

In [1]:
import functions as dlf
import pandas as pd
import numpy as np
import requests
import regex
import pickle
from importlib import reload
from bs4 import BeautifulSoup
from genderize import Genderize

In [66]:
import numpy as np
# import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import tzlocal
import datetime as dt
import xgboost as xgb
import sklearn
import sklearn.metrics as metrics
from yellowbrick.classifier import ROCAUC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from imblearn.over_sampling import SMOTE

%matplotlib inline

In [ ]:
# with open('election_collection_raw.pickle', 'wb') as f:
#     pickle.dump(election_collection_raw, f)
#     f.close()

# with open('election_collection_raw.pickle', 'rb') as f:
#     election_collection_raw = pickle.load(f)
#     f.close()

In [71]:
reload(dlf)

<module 'functions' from 'C:\\Users\\d_ful\\Documents\\GitHub\\Capstone_Project\\functions.py'>

In [62]:
df = pd.read_csv('Senate_generals_thru_2016')
df_gend = df.copy()
df_model_test = df.copy()
# display(df.head(10))
display(df.tail())
# df.info()

,%,Turnout,Incumb_Y,State,Cln_name,Year,Terms_in_office,Party_enc,First_name,Seats_up%,Seats_before%
5583,1.01,320467.0,0,Vermont,Peter Diamondstone,2016,0,T,Peter,0.000000,0.00
5584,59.01,3243317.0,1,Washington,Patty Murray,2016,4,D,Patty,0.227273,0.44
5585,40.99,3243317.0,0,Washington,Chris Vance,2016,0,R,Chris,0.444444,0.54
5586,50.20,2947345.0,1,Wisconsin,Ron Johnson,2016,1,R,Ron,0.444444,0.54
5587,46.80,2947345.0,0,Wisconsin,Russ Feingold,2016,0,D,Russ,0.227273,0.44


In [63]:
## Drop name refs
df_model_test.drop(columns=['Cln_name', 'First_name'], inplace=True)

## Fill NAs left from cleaning
df_model_test.dropna(inplace=True)

## Fixing State regex missed cases
df_model_test.at[1461, 'State'] = 'West_Virginia'
df_model_test.at[1632, 'State'] = 'Nevada'

## Setting variables + Q.C.
y = df_model_test.pop('%')
X = df_model_test.copy()
display(y.head())
display(X.head())
display(X.info())

## drop_first assumes that State = Alabama and Party = D
X_ohe = pd.get_dummies(X, drop_first=True)
display(X_ohe.columns)

## TT split
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, random_state=222)

0    66.10
1    33.05
2     0.85
3    69.27
4    29.51
Name: %, dtype: float64

,Turnout,Incumb_Y,State,Year,Terms_in_office,Party_enc,Seats_up%,Seats_before%
0,233985.0,1,Alabama,1920,1,D,0.361702,0.489583
1,233985.0,0,Alabama,1920,0,R,0.306122,0.510417
2,233985.0,0,Alabama,1920,0,S,0.000000,0.000000
3,231960.0,0,Alabama,1920,0,D,0.361702,0.489583
4,231960.0,0,Alabama,1920,0,R,0.306122,0.510417


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5583 entries, 0 to 5587
Data columns (total 8 columns):
Turnout            5583 non-null float64
Incumb_Y           5583 non-null int64
State              5583 non-null object
Year               5583 non-null int64
Terms_in_office    5583 non-null int64
Party_enc          5583 non-null object
Seats_up%          5583 non-null float64
Seats_before%      5583 non-null float64
dtypes: float64(3), int64(3), object(2)
memory usage: 392.6+ KB


None

Index(['Turnout', 'Incumb_Y', 'Year', 'Terms_in_office', 'Seats_up%',
       'Seats_before%', 'State_Alaska', 'State_Arizona', 'State_Arkansas',
       'State_California', 'State_Colorado', 'State_Connecticut',
       'State_Delaware', 'State_Florida', 'State_Georgia', 'State_Hawaii',
       'State_Idaho', 'State_Illinois', 'State_Indiana', 'State_Iowa',
       'State_Kansas', 'State_Kentucky', 'State_Louisiana', 'State_Maine',
       'State_Maryland', 'State_Massachusetts', 'State_Michigan',
       'State_Minnesota', 'State_Mississippi', 'State_Missouri',
       'State_Montana', 'State_Nebraska', 'State_Nevada',
       'State_New_Hampshire', 'State_New_Jersey', 'State_New_Mexico',
       'State_New_York', 'State_North_Carolina', 'State_North_Dakota',
       'State_Ohio', 'State_Oklahoma', 'State_Oregon', 'State_Pennsylvania',
       'State_Rhode_Island', 'State_South_Carolina', 'State_South_Dakota',
       'State_Tennessee', 'State_Texas', 'State_Utah', 'State_Vermont',
       'State_

In [106]:
## Instantiating a bunch of classifiers to test

# tree_clf = DecisionTreeRegressor()
# rf_clf = RandomForestRegressor(n_jobs=-1)
# knn_clf = KNeighborsRegressor(n_jobs=-1)
# ada_clf = AdaBoostRegressor()
# grad_clf = GradientBoostingRegressor()
xgb_clf = xgb.XGBRegressor(n_jobs=-1)

In [107]:
# y_hat_trn, y_hat_te = dlf.fit_n_pred(tree_clf, X_train, X_test, y_train)
# y_hat_trn, y_hat_te = dlf.fit_n_pred(rf_clf, X_train, X_test, y_train)
# y_hat_trn, y_hat_te = dlf.fit_n_pred(knn_clf, X_train, X_test, y_train)
# y_hat_trn, y_hat_te = dlf.fit_n_pred(ada_clf, X_train, X_test, y_train)
# y_hat_trn, y_hat_te = dlf.fit_n_pred(grad_clf, X_train, X_test, y_train)
y_hat_trn, y_hat_te = dlf.fit_n_pred(xgb_clf, X_train, X_test, y_train)

---- Timer stopped at: 08/20/2020 - 08:27 PM ----
---- Time elasped: 0:00:01.192832 ----


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

In [76]:
## Tree_clf
print('Training MSE:', metrics.mean_squared_error(y_true=y_train, y_pred=y_hat_trn))
print('Test MSE:', metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_te))

Training MSE: 10.350590170590376
Test MSE: 162.38372754168535


In [79]:
## rf_clf
print('Training MSE:', metrics.mean_squared_error(y_true=y_train, y_pred=y_hat_trn))
print('Test MSE:', metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_te))

Training MSE: 22.711305203633113
Test MSE: 119.3454431891556


In [99]:
## knn_clf
print('Training MSE:', metrics.mean_squared_error(y_true=y_train, y_pred=y_hat_trn))
print('Test MSE:', metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_te))

Training MSE: 491.8248684267953
Test MSE: 746.8824784133737


In [102]:
## ada_clf
print('Training MSE:', metrics.mean_squared_error(y_true=y_train, y_pred=y_hat_trn))
print('Test MSE:', metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_te))

Training MSE: 204.2453250875433
Test MSE: 223.0358615293897


In [105]:
## grad_clf
print('Training MSE:', metrics.mean_squared_error(y_true=y_train, y_pred=y_hat_trn))
print('Test MSE:', metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_te))

Training MSE: 99.0635530788416
Test MSE: 129.40290622653492


In [108]:
## xgb_clf
print('Training MSE:', metrics.mean_squared_error(y_true=y_train, y_pred=y_hat_trn))
print('Test MSE:', metrics.mean_squared_error(y_true=y_test, y_pred=y_hat_te))

Training MSE: 101.36131548183157
Test MSE: 129.3823463947934


In [92]:
display(X_train.head())
display(y_train.head())

,Turnout,Incumb_Y,Year,Terms_in_office,Seats_up%,Seats_before%,State_Alaska,State_Arizona,State_Arkansas,State_California,...,State_Vermont,State_Virginia,State_Washington,State_West_Virginia,State_Wisconsin,State_Wyoming,Party_enc_I,Party_enc_R,Party_enc_S,Party_enc_T
499,1414440.0,1,1928,2,0.255319,0.500000,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,191858.0,0,1920,0,0.306122,0.510417,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4345,2860157.0,0,1996,0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1735,2055798.0,0,1948,0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
715,306611.0,1,1930,3,0.333333,0.406250,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


499     55.30
9       34.08
4345     0.40
1735     0.45
715     86.93
Name: %, dtype: float64

In [93]:
X.loc[499]

Turnout            1.41444e+06
Incumb_Y                     1
State                  Indiana
Year                      1928
Terms_in_office              2
Party_enc                    R
Seats_up%             0.255319
Seats_before%              0.5
Name: 499, dtype: object

In [96]:
y_hat_trn[0]

58.19450000000003

In [86]:
pd.Series(y_hat_trn).value_counts()

3.528037     8
0.165362     7
1.605361     6
0.348780     6
1.558649     6
7.457029     5
0.840185     5
0.795742     5
0.840021     5
0.474410     5
0.530851     5
8.254186     5
0.177688     5
0.212224     4
0.474689     4
12.430022    4
0.209002     4
4.467058     4
1.765107     4
0.746054     4
0.919497     4
0.913622     4
1.186195     4
0.108528     4
0.291081     4
0.028437     4
1.155841     4
1.431930     4
0.307633     4
0.628453     4
            ..
46.926500    1
53.882200    1
48.116000    1
80.653100    1
2.640267     1
50.593700    1
0.551020     1
52.931700    1
66.979600    1
48.456200    1
56.351000    1
0.896300     1
49.693300    1
51.202200    1
52.943600    1
41.511100    1
39.287900    1
48.122200    1
49.547500    1
17.853933    1
22.718102    1
38.883400    1
50.963500    1
53.262700    1
60.471100    1
0.679610     1
42.411500    1
47.396181    1
36.485150    1
59.119500    1
Length: 3634, dtype: int64

## Gender stuff

In [22]:
def gender_guesser(inp_df, count_max=1000, g_thresh=0.6, online=False):
    import pandas as pd
    import numpy as np
    from genderize import Genderize
    
    ## Instantiate Genderize + making varibles for use
    g_izer = Genderize()
    out_df = inp_df.copy()
    count = 0
    
    ## Q.C. for formatting
    if 'Gender' not in out_df.columns:
        out_df['Gender'] = 'None'

    ## Looping through ea. name and making a guess
    for idx, name in zip(out_df.index, out_df['First_name']):
        ## Filtering out previously filled items
        if out_df['Gender'].loc[idx] == 'None':
            pred = None
            ## Try statement for api error results
            try:
                ## Pull guess dict from api or manufacture one
                if online:
                    guess = g_izer.get([name])[0]
                else:
                    guess = {}
                    guess['gender'] = np.random.choice(['male', 'female'])
                    guess['probability'] = np.random.randint(0.01, high=1)

                ## Executing custom threshold for gender probability
                if guess['gender'] == 'male':
                    if guess['probability'] < g_thresh:
                        pred = 'F'
                    else:
                        pred = 'M'

                if guess['gender'] == 'female':
                    if guess['probability'] < g_thresh:
                        pred = 'M'
                    else:
                        pred = 'F'

                ## Setting the prediction into the correct index
                out_df.at[idx, 'Gender'] = pred
                count += 1
                
            except Exception as e: #GenderizeException:
                print('Pred Error:\n')
                print(e, '\n')
                break
        
        ## Pseudo call limit for offline testing
        if (1 < count) & (count % count_max == 0):
            print(('***'*10), f'Hit {count_max} name limit.', ('***'*10))
            return out_df
        
    print(('***'*10),f'Finished {count} names. Remaining available: {count_max-count}.',('***'*10))
    return out_df

In [23]:
df_gend = gender_guesser(df_gend, online=True)

Pred Error:

('Request limit reached', 429, {'Server': 'nginx/1.16.1', 'Date': 'Fri, 21 Aug 2020 01:46:14 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '33', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET', 'Access-Control-Allow-Headers': 'Content-Type, X-Genderize-Source', 'X-Rate-Limit-Limit': '1000', 'X-Rate-Reset': '80026', 'X-Rate-Limit-Remaining': '0', 'ETag': 'W/"21-tYoIBroDGdB+35cIAOMCdpXfqjI"'}) 

****************************** Finished 0 names. Remaining available: 1000. ******************************


In [ ]:
df_gend['Gender'].value_counts()

In [101]:
df_gend['Gender'].value_counts()

None    3588
M       1844
F         94
Name: Gender, dtype: int64

In [104]:
df_gend['Gender'].value_counts()

M       2757
None    2588
F        150
Name: Gender, dtype: int64

In [106]:
df_gend['Gender'].value_counts()

M       3616
None    1588
F        259
Name: Gender, dtype: int64

In [109]:
df_gend['Gender'].value_counts()

M       4448
None     588
F        406
Name: Gender, dtype: int64

In [151]:
holder_m = ['James Eli Watson', 'Selden Spencer', 'Forsman', 'Trudell', 
          'Robert La Follette', 'Cherry', 'France', 'Eickelberg', 
          'Day', 'Kindler', 'Carey', 'Duly McCone', 'Huey Long', 'Lanier', 
          'Glen Corkery', 'Starr', 'Hager', 'Verhei', 'Happy Chandler', 
          'Philo Farnsworth', 'Ona Searles', 'Lewis Haley Tiley', 'Cecil Creel',
          'John Marion Morris', 'Lynn Frazier', 'Glen Taylor', 'Hanna',
          'Lynn Stumbaugh', 'Alexis DuPont Bayard', 'Arthur Vivian Watkins',
          'Val Bjornson', 'Laurence Pickett', 'Lou Welch', 'John Little McClellan',
          'Cecil Underwood', 'Melvin Babcock Morgan', 'Cecil Heftel', 'Cecil Pitard',
          'Laurence Burton', 'Lynn Henderson', 'Lorin Oxley', 'Chic Hecht',
          'Arlyn Wollenburg', 'Lou Gold', 'Mel Logan', 'Lindsey Graham',
          'Lamar Alexander', 'John Neely Kennedy', 'Maclyn Stringer', 'Cam Cavasso',
          'LeAlan Jones', 'Rand Paul', 'Tracy Potter', 'David Lory VanDerBeek',
          'Paul Lindsey Sadler']

holder_n = ['Miscellaneous', 'Others', 'Write-In Candidates']

holder_c = ['Lawrence Gilfedder,Candidate for Governor in 1948. Ran for Governor in 1952 and 1954. Ran for Senate in 1958, 1960, 1962, 1964, 1966, and 1970.',
            'Jerry Zeldin, Laurel Springs', 'Howard Cannon, City Attorney of Las Vegas',
            'Douglas McKay,former Governor of Oregon  and United States Secretary of the Interior',
            'Efia Nwangaza*']

for i, name in zip(test.index, test['Cln_name']):
    for target in holder_m:
        if name == target:
            test.at[i, 'Gender'] = 'M'
            
    for target in holder_n:
        if name == target:
            test.at[i, 'Gender'] = 'None'
            
    if name == holder_c[0]:
        n_name = 'Lawrence Gilfedder'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[1]:
        n_name = 'Jerry Zeldin'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[2]:
        n_name = 'Howard Cannon'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[3]:
        n_name = 'Douglas Mckay'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[4]:
        n_name = 'Efia Nwangaza'
        test.at[i, 'Cln_name'] = n_name
            

In [5]:
test = df_gend[df_gend['Gender'] == 'F'].copy()
test = test[test['Gender'] == 'F'].copy()
print(len(test['Gender']), '\n')
for name in test['Cln_name']:
    print(name)


KeyError: 'Gender'

In [154]:
test[test['Turnout'].isna()]

,%,Turnout,Incumb_Y,State,Cln_name,Year,Terms_in_office,Party_enc,First_name,Seats_up%,Seats_before%,Gender
4406,100.0,NaN,0,Arkansas,Ophie Beltran,1998,0,T,Ophie,0.0,0.0,F


## Next..

In [ ]:
## Make gender collection function #####################
## pickle/to_csv funct/df to Yish for help #############
## Keep year? Set_index as year (as datetime)
## Split % off as target
## OHE States, Party_enc
## RF/Tree Regressors 1st XGBoost?
## add first_name columns into df_fin ##################
## Make sure Turnout is ok as int. #####################

In [95]:
# test_df = df[df['Year'] == 2006].copy()
# display(test_df.head())
# display(test_df.info())

In [94]:
# test_df = gender_guesser(test_df, count_max=50)
# display(test_df.head())
# display(test_df.tail())